被操作的对象必须是一个张量，因此需要把标量转化为一个0维张量，形参是对实参地址的指针。
这种不显式分配GPU内存的方法，会由于数据在CPU和GPU之间来回搬运造成计算能力的损耗
在下面这个代码块中，numba自动完成了以下事情：
+ 编译了一个CUDA内核，在所有的输入元素上并行地执行ufunc操作。
+ 为输入和输出分配GPU内存。
+ 将输入数据复制到GPU上。
+ 根据输入的大小，以正确的内核尺寸执行CUDA内核。
+ 将结果从GPU复制到CPU。
+ 将结果以NumPy数组的形式返回到主机上。

In [71]:
import numba
import numpy as np
from numba import cuda


@cuda.jit
def check_prime_gpu_kernel(num, result):
    for i in range(2, num // 2 + 1):
        if (num % i) == 0:
            result[0] = 0
    else:
        result[0] = num


def find_all_primes_cpu_and_gpu(upper):
    all_prime_numbers = []
    for num in range(2, upper):
        result = np.zeros((1), np.int32)
        # 在下步中完成数据cpu->gpu、GPU计算、数据gpu->cpu。此外[1, 1]有着特殊含义
        check_prime_gpu_kernel[1, 1](num, result)

        if result[0] > 0:
            all_prime_numbers.append(num)
    return all_prime_numbers

In [72]:
find_all_primes_cpu_and_gpu(10)

[2, 3, 4, 5, 6, 7, 8, 9]

利用ufunc进行计算加速。下面代码首先使用cpu并行进行加速

In [239]:
# 定义一个ufunc
import math
import numba
from numba import vectorize


@numba.jit(nopython=True,
           fastmath=True,
           nogil=True)
def is_prime(num):
    flag = True
    for i in range(2, math.floor(math.sqrt(num)) + 1):
        if (num % i) == 0:
            flag = False
            break
    return flag


@vectorize(['int32(int32)'], target='cpu')
def check_prime(num):
    flag = 0
    if is_prime(num):
        flag = 1
    return flag

In [240]:
import numpy as np
n = 10_000_000
test_np_array = np.arange(2, n)
%timeit -r 1 -n 5 out = check_prime(test_np_array)

5.15 s ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


下面代码再使用在gpu上的ufunc进行加速，加速的方法包含有：
+ `@numba.cuda.jit(device=True)`指定cuda上操作
+ `@vectorize([<type_expr>], target='cuda')`指定ufunc在cuda上操作
+ ufunc操作的对象由`cupy`已经**提前**且**直接**创建在cuda上了

值得注意的是，利用cuda计算的函数内部只能包含以下操作：
+ `if`/`elif`/`else`
+ `while` and `for` loops
+ Basic math operators
+ Selected functions from the `math` and `cmath` modules
+ Tuples

需要明确的是，在cuda上进行的操作最好的方法是，先开辟一块内存空间后（开辟空间的计算消耗较大），直接在已开辟的空间上进行数值运算。
同时，对于输出，最好能够**直接将计算结果保存在一块已经开辟的内存空间内**。这样能够以`cupy`的`array`对象保存操作结果。
相较于cpu上的计算，已经达到20~30倍的加速了。
相较于cpu上的计算，已经达到20~30倍的加速了。

In [2]:
import math
from numba import vectorize, cuda


@cuda.jit(device=True)
def is_prime(num):
    flag = True
    for i in range(2, math.floor(math.sqrt(num)) + 1):
        if (num % i) == 0:
            flag = False
            break
    return flag


@vectorize(['int32(int32)'], target='cuda')
def check_prime(num):
    flag = 0
    if is_prime(num):
        flag = 1
    return flag

In [5]:
import cupy as cp
n = 10_000_000
test_cp_array = cp.arange(2, n)
out_cp_array = cp.empty_like(test_cp_array)
%timeit -r 1 -n 5 out = check_prime(test_cp_array)
%timeit -r 1 -n 5 out = check_prime(test_cp_array, out=out_cp_array) # 速度上没有什么变化

221 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)
226 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


接下来考虑多种在gpu上分配内存的方法。
显然，利用cupy直接在gpu上分配的速度最快。

In [288]:
import numpy as np
import cupy as cp
import numba
n = 100_000_000
%timeit -r 1 -n 5 a_device = numba.cuda.to_device(np.arange(n)) # cpu -> gpu via numba
%timeit -r 1 -n 5 b_device = cp.asarray(np.arange(n)) # cpu -> gpu via cupy
%timeit -r 1 -n 5 c_empty_device = numba.cuda.device_array(n) # like np.empty() via numba
%timeit -r 1 -n 5 d_empty_device = cp.empty(n) # like np.empty() via cupy
%timeit -r 1 -n 5 e_device = cp.arange(n) # allocate directly

301 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)
210 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)
42.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)
982 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)
98.1 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 5 loops each)


下面的代码对GPU内存进行释放

In [1]:
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()
print(mempool.total_bytes(), "Bytes")
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()
print(mempool.total_bytes(), "Bytes")

NameError: name 'cp' is not defined